# Sample Steam Reviews with GPT-2
Code inspired from https://github.com/woctezuma/sample-steam-reviews-with-gpt-2

## Setting the GPT-2 model

Install the Python package

Reference: https://github.com/minimaxir/gpt-2-simple

In [0]:
!pip install gpt_2_simple

Download the pre-trained model

In [0]:
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## Downloading GPT-2

In [0]:
gpt2.download_gpt2()

Fetching checkpoint: 1.00kit [00:00, 827kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 47.6Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 317kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 66.1Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 1.46Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 35.4Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 33.8Mit/s]                                                       


## Mounting Google Drive

In [0]:
gpt2.mount_gdrive()

## Uploading a Text File to be Trained to Colaboratory

#### Either get the data by yourself

In [0]:
!curl -O https://raw.githubusercontent.com/woctezuma/sample-steam-reviews-with-gpt-2/master/export_review_data.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5303  100  5303    0     0  43467      0 --:--:-- --:--:-- --:--:-- 43467


In [0]:
!curl -O https://raw.githubusercontent.com/woctezuma/sample-steam-reviews-with-gpt-2/master/requirements.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    37  100    37    0     0    313      0 --:--:-- --:--:-- --:--:--   313


In [0]:
!pip install -r requirements.txt

    100% |████████████████████████████████| 1.0MB 23.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
app_id = 583950

In [0]:
from export_review_data import apply_workflow_for_app_id

apply_workflow_for_app_id(app_id)

A command-line argument is detected. AppID set to 583950.
[appID = 583950] expected #reviews = 8701
#reviews = 180
Filtering out reviews which were not written in english.
#reviews = 180
Filtering out reviews with strictly fewer than 150 characters.
#reviews = 101
Filtering out reviews which were not detected as written in en.
#reviews = 101


#### Or get a data snapshot from me

In [0]:
#!curl -O https://raw.githubusercontent.com/woctezuma/sample-steam-reviews-with-gpt-2/master/data/583950.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  118k  100  118k    0     0   758k      0 --:--:-- --:--:-- --:--:--  758k


## Finetune GPT-2

In [0]:
file_name = str(app_id) + '.txt'

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              run_name='reviews',
              dataset=file_name,
              steps=1000,
              restore_from='fresh',   # change to 'latest' to resume training
              print_every=10,   # how many steps between printing progress
              sample_every=200,   # how many steps to print a demo sample
              save_every=500   # how many steps between saving checkpoint              
              )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Loading checkpoint models/117M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  3.46it/s]


dataset has 27698 tokens
Training...
[10 | 29.03] loss=3.19 avg=3.19
[20 | 51.91] loss=3.12 avg=3.16
[30 | 75.73] loss=2.13 avg=2.81
[40 | 99.69] loss=1.83 avg=2.56
[50 | 123.14] loss=1.73 avg=2.39
[60 | 146.52] loss=1.22 avg=2.19
[70 | 170.17] loss=0.85 avg=2.00
[80 | 193.87] loss=0.47 avg=1.80
[90 | 217.50] loss=0.23 avg=1.62
Saving checkpoint/run1/model-100


In [0]:
gpt2.copy_checkpoint_to_gdrive()

## Load a Trained Model Checkpoint

In [0]:
gpt2.copy_checkpoint_from_gdrive()

In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess,
               run_name='reviews')

## Generate Text From The Trained Model

In [0]:
num_samples = 3
num_batches = 3 # Unique to GPT-2, you can pass a batch_size to generate multiple samples in parallel, giving a massive speedup.

gpt2.generate(sess,
              nsamples=num_samples,
              batch_size=num_batches)

A common thread runs throughout all of this: Why not just buy the game from the start? And what if the outcome of the game weren't so damn clear cut? What if Honest Abe failed? What if Honest Abe was somehow fundamentally flawed?
Let me put it this way, if Honest Abe failed, how would you justify buying a game if you actually got the game...
You guessed it, it actually was better than the fucking game you just purchased.
Seriously, though, what would you buy if you were only going to be playing for a few minutes?
"Holy cow that was better than the fucking game you actually bought..."
"Holy cow that was better than the card game you actually played...wait for the answer...it's literally the WORST PAIR IN THE ENTIRE GAME"
I know, mentioned just to name some players. But damn did I hope that all the good cards, sounds and mechanics would be here in this game. Not just one or two, but three or four.
If you're the best player in the game, why can't you play for free? Don't you want to throw

In [0]:
gpt2.generate(sess,
              nsamples=num_samples,
              batch_size=num_batches,
              prefix='I love Artifact')

I love Artifact, I played it in draft on rng, I won the premade 8k MMR with it and am going to continue to do so.
Artifact is fun, plays pretty sweet game with a decent amount of microtransactions, there are perks that you can get that are worth it, and the fact that you have to open packs to access them, that makes me a hardcore CCG player.
However, I think the point of this game is to throw cards out the window, and have a casual game for kids. That is all.
TL;DR - I like Artifact, I played it in draft on rng, I won the premade 8k MMR with it and am going to continue to do so.

P.S. If you've made it this far, thank you so much for reading my story, and I hope you'll agree that I am an interesting person with a passion for card games and DC. I look forward to learning more about your business as usual!
As always, I appreciate your time and appreciate the efforts!
As always, I will be adding as much as I can as soon as I have time, so if you have time, please do! :)
Cheers,
The Man
(V

In [0]:
gpt2.generate(sess,
              nsamples=num_samples,
              batch_size=num_batches,
              prefix='I hate Artifact')

I hate Artifact. I despise the idea of a good hero ever being able to expand his or her library, even if he or she ultimately does, I don't see a way for this to happen here.
The three lane thing might be cool, but how do you make an opponent understand that you have three cards that can potentially affect the game? When it comes to deciding the pros and cons of each, there's just not a lot to discuss. There's just a single card that fundamentally affects the game in its current state. Even after adding and adjusting cards, an opponent still won't be able to undo the damage that was done to their deck due to only one card. This is one of the biggest reasons why Artifact has gotten so popular, it's hard to ignore the fact that it is one of the best cards series in existence.
Having said all that, I'm a big RNG fan. Every game I do ends in a draw of the three hero round, which is utterly infuriating and out of my 10 on this game, it's probably the least infuriating. Artifact is a neat ca

In [0]:
gpt2.generate(sess,
              nsamples=num_samples,
              batch_size=num_batches,
              prefix='Please, Valve')

P, Valve give us a fair shake, because for all of you saying that this game doesn't have much depth or variety, it's actually pretty deep and diverse.  There are a lot of skills to learn, and a lot of ways to learn to master them.  Some of them are the strongest: bluff, pass, etc.
However, there are some skills that really stand out.  One of them is her incredible multitasking.  She can do almost anything.  Can edit her profile, change her spells, add her subclasses, etc.
Her unique deck building mechanics are awesome.  I have two main drafts left in the draft game.  I have a draft chamber that I can play cards from, and a bunch of other cards that I can trade with friends.  Some of them are just amazing: salvaging cards drawn from your library for new spells, artifacts, etc.
Rng neutralize+foe+ignition+the enemy can copy these abilities and play with them.  And in some cases, even bring them as an add-on.  It's a whole lot of fun.
But just for the sake of argument and argument (as muc